In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import implicit
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

# 1. load data

In [27]:
buy = pd.read_csv('../data/buy_data.csv') # 구매이력
history = pd.read_csv('../data/history_data.csv') # 시청 시작 데이터
watch = pd.read_csv('../data/watch_e_data.csv') # 시청 종료 데이터
meta = pd.read_csv('../data/meta_data.csv') # 콘텐츠 일반 메타
meta_plus = pd.read_csv('../data/meta_data_plus.csv') # 콘텐츠 확장 정보
profile = pd.read_csv('../data/profile_data.csv') # 프로필 정보
search = pd.read_csv('../data/search_data.csv') # 검색을 통한 시청 데이터
sample = pd.read_csv('../data/sample_submission.csv')
history['watch'] = 1

c:\users\y0010\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
his_group = history.groupby(['profile_id','album_id']).sum().reset_index()
his_group

,profile_id,album_id,ss_id,log_time,payment,watch
0,3,15,20220301115653,20220301115719,0.0,1
1,3,16,20220301115653,20220301115809,0.0,1
2,3,17,20220301115653,20220301115958,0.0,1
3,3,18,20220301115653,20220301120118,0.0,1
4,3,19,20220301115653,20220301120229,0.0,1
...,...,...,...,...,...,...
380542,33032,7105,424628959523801,424628959623429,0.0,21
380543,33032,8910,20220426115550,20220426130410,0.0,1
380544,33032,14662,20220427112116,20220427112335,0.0,1
380545,33032,14663,40440854224232,40440854224753,0.0,2


In [29]:
customers = list(np.sort(his_group['profile_id'].unique()))
album = list(his_group['album_id'].unique())
watch = list(his_group['watch'])

rows = his_group['profile_id'].astype('category').cat.codes
cols = his_group['album_id'].astype('category').cat.codes
print(len(customers)) 
print(len(album))  
purchase_sparse = sparse.csr_matrix((watch, (rows, cols)), shape = (len(customers),len(album)))
purchase_sparse 

8311
20695


<8311x20695 sparse matrix of type '<class 'numpy.intc'>'
	with 380547 stored elements in Compressed Sparse Row format>

In [30]:
# Sparsity: 얼마나 비어있나?
matrix_size = purchase_sparse.shape[0]* purchase_sparse.shape[1]
num_purchases = len(purchase_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases / matrix_size))
sparsity

99.77874678528406

In [31]:
als_model = ALS(factors=20, regularization=0.01, iterations = 100)
als_model.fit(purchase_sparse)

In [13]:
customers[8300:]

[32998, 33000, 33011, 33015, 33016, 33019, 33022, 33023, 33026, 33027, 33032]

In [34]:
result = []
for i in range(len(customers)):
    result.append(list(als_model.recommend(i, purchase_sparse[i], N=25)[0]))

In [35]:
pd.Series(result)

0       [38, 222, 223, 39, 227, 134, 228, 40, 239, 949...
1       [327, 342, 471, 963, 357, 36, 1791, 398, 1961,...
2       [1791, 239, 1963, 342, 327, 357, 122, 0, 123, ...
3       [398, 401, 3796, 399, 400, 404, 403, 60, 402, ...
4       [123, 188, 125, 239, 49, 53, 124, 1938, 50, 17...
                              ...                        
8306    [186, 1376, 54, 1397, 1529, 1891, 2105, 2564, ...
8307    [335, 125, 239, 49, 124, 53, 1938, 50, 1791, 3...
8308    [2656, 2665, 2664, 2666, 2655, 2668, 2654, 266...
8309    [122, 63, 123, 188, 335, 125, 239, 1938, 53, 3...
8310    [122, 125, 239, 49, 124, 50, 370, 327, 53, 342...
Length: 8311, dtype: object

In [36]:
sample['predicted_list']= result

In [38]:
sample.to_csv('result2.csv',index=None)

In [15]:
his_group

,profile_id,album_id,ss_id,log_time,payment
0,3,15,20220301115653,20220301115719,1
1,3,16,20220301115653,20220301115809,1
2,3,17,20220301115653,20220301115958,1
3,3,18,20220301115653,20220301120118,1
4,3,19,20220301115653,20220301120229,1
...,...,...,...,...,...
380542,33032,7105,424628959523801,424628959623429,1
380543,33032,8910,20220426115550,20220426130410,1
380544,33032,14662,20220427112116,20220427112335,1
380545,33032,14663,40440854224232,40440854224753,1
